In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

from sklearn.metrics import root_mean_squared_error, mean_squared_error

In [5]:
import mlflow

mlflow.set_tracking_uri("sqlite:///new_mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/ganesh/Documents/mlops_zoomcamp/mlops-zoomcamp/02-Experiment-tracking/mlruns/1', creation_time=1716913801964, experiment_id='1', last_update_time=1716913801964, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [6]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [7]:
df_train = read_dataframe("./data/yellow_tripdata_2023-01.parquet")
df_val = read_dataframe("./data/yellow_tripdata_2023-02.parquet")

In [8]:
len(df_train), len(df_val)

(3009173, 2855951)

In [11]:
# df_train.duration.describe

In [8]:
df_train["PU_DO"] = df_train["PULocationID"] + '_' + df_train["PULocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + '_' + df_val["PULocationID"]

In [13]:
df_val ;

In [9]:
categorical = ['PULocationID','DOLocationID'] #["PU_DO"]
#numerical = ['trip_distance']

dv = DictVectorizer()

# train_dicts = df_train[categorical + numerical].to_dict(orient = "records")
train_dicts = df_train[categorical].to_dict(orient = "records")
X_train = dv.fit_transform(train_dicts)


In [10]:
# val_dicts = df_val[categorical + numerical].to_dict(orient = "records")
val_dicts = df_val[categorical].to_dict(orient = "records")
X_val = dv.transform(val_dicts)

In [11]:
target= 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [17]:
X_val[0].toarray().shape

(1, 515)

In [18]:
X_train[0].toarray().shape
# X_train[0].toarray()

(1, 515)

In [17]:
# testing model using jan data, and using feb data for validation(testing)
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)


In [18]:
root_mean_squared_error(y_val, y_pred)

7.811817942871299

In [19]:
with open('./models/lin_reg.bin', 'wb' ) as f_out :
    pickle.dump((dv, lr), f_out)

In [20]:
# testing model with its own traning data as validation data
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
root_mean_squared_error(y_train, y_pred)

7.649261952010433

In [26]:
# testing model with its traning data as validation data
with mlflow.start_run():

    mlflow.set_tag("developer", "Ganesh")

    mlflow.log_param("train-data-path", "./data/yellow_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path", "./data/yellow_tripdata_2023-02.parquet")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="./models/lin_reg.bin", artifact_path="models_pickle")

In [27]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [28]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [29]:

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=500,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [34]:
with mlflow.start_run():
    params = {"learning_rate":0.7727401314948932,
                "max_depth":63,
                "min_child_weight":6.474336231446163,
                "objective" : "reg:linear",
                "reg_alpha" :0.09158212128338573,
                "reg_lambda":0.004014057311104894,
                "seed" :42, 
    }

    mlflow.log_params(params)
        
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=500,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open('./models/preprocessor.b', 'wb' ) as f_out :
        pickle.dump((dv, lr), f_out)
    
    mlflow.log_artifact('./models/preprocessor.b', artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/ganesh/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:19:28] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1713397725960/work/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:7.26796
[1]	validation-rmse:6.00607
[2]	validation-rmse:5.87216
[3]	validation-rmse:5.68468
[4]	validation-rmse:5.62628
[5]	validation-rmse:5.54871
[6]	validation-rmse:5.45859
[7]	validation-rmse:5.36195
[8]	validation-rmse:5.32371
[9]	validation-rmse:5.31609
[10]	validation-rmse:5.31253
[11]	validation-rmse:5.29878
[12]	validation-rmse:5.27977
[13]	validation-rmse:5.27414
[14]	validation-rmse:5.27180
[15]	validation-rmse:5.27047
[16]	validation-rmse:5.26934
[17]	validation-rmse:5.26634
[18]	validation-rmse:5.26522
[19]	validation-rmse:5.26428
[20]	validation-rmse:5.26364
[21]	validation-rmse:5.26231
[22]	validation-rmse:5.26182
[23]	validation-rmse:5.26136
[24]	validation-rmse:5.26100
[25]	validation-rmse:5.26059
[26]	validation-rmse:5.25971
[27]	validation-rmse:5.25932
[28]	validation-rmse:5.25879
[29]	validation-rmse:5.25848
[30]	validation-rmse:5.25775
[31]	validation-rmse:5.25740
[32]	validation-rmse:5.25751
[33]	validation-rmse:5.25723
[34]	validation-rmse:5.2

/home/ganesh/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/ganesh/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:38:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1713397725960/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [35]:
logged_model = 'runs:/41132515eb5149bca0c3504f1c81cf71/models_mlflow'

In [36]:
xgb_model = mlflow.xgboost.load_model(logged_model)

/home/ganesh/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:39:27] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1713397725960/work/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [38]:
y_pred = xgb_model.predict(valid)

array([ 6.3284273, 44.159    , 15.9019   , ..., 15.249503 , 11.978197 ,
       13.525034 ], dtype=float32)

In [39]:
y_pred[:10]

array([ 6.3284273, 44.159    , 15.9019   , 21.602337 , 25.264547 ,
        5.5126605, 20.113096 ,  5.9981194,  6.8515706, 12.069719 ],
      dtype=float32)